In [ ]:
# Set up key libraries / tokens

In [9]:
import requests 
from requests_oauthlib import OAuth1
import os
import pandas as pd

In [10]:
API_PUBLIC = os.environ.get("TW_API_KEY")
API_SECRET = os.environ.get("TW_API_SECRET_KEY")
APP_PUBLIC = os.environ.get("TW_PRS_GA_HW_ACCESS_TOKEN")
APP_SECRET = os.environ.get("TW_PRS_GA_HW_ACCESS_TOKEN_SECRET")

In [11]:
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
auth = OAuth1(
            API_PUBLIC, #'USER_OAUTH_TOKEN', 
            API_SECRET, #'USER_OAUTH_TOKEN_SECRET'
            APP_PUBLIC, #'YOUR_APP_KEY', 
            APP_SECRET #'YOUR_APP_SECRET',
            )

In [12]:
# Check authorisation is working properly

In [13]:
check_auth = requests.get(url, auth=auth)

In [14]:
check_auth.reason

'OK'

In [15]:
check_auth.status_code

200

# Function 5

Rewrite the `friends_of_friends` function, except this time include an argument called `full_search`, which accepts a boolean value.  If set to `True`, use cursoring to cycle through the complete set of users for the users provided.  

The twitter API only returns a subset of users in your results to save bandwidth, so you have to cycle through multiple result sets to get all of the values.

You can read more about how this works here:  https://developer.twitter.com/en/docs/basics/cursoring

Basically you have to do a `while` loop to continually make a new request using the values stored in the `next_cursor` key as part of your next query string until there's nothing left to search.

**Note:** We're using the free API, so we're operating under some limitations.  One of them being that you can only make 15 API calls in a 15 minute span to this portion of the API.  You can also only return up to 200 results per cursor, so this means you won't be able to completely search for everyone even if you set this up correctly.

That's fine, just do what you can under the circumstances.

**To Test:** To test your function, we'll run the following function calls:

 - `friends_of_friends(['ezraklein', 'tylercowen'])` -- should return 4 results if you do an API call that returns 200 results
 - `friends_of_friends(['ezraklein', 'tylercowen'], full_search=True)` -- should return 54 results if you do an API call that returns 200 results
 
**Hint:** Chances are you will exhaust your API limits quite easily in this function depending on who you search for.  Depending on how you have things set up, this could cause error messages to arise when things are otherwise fine.  Remember in class 3 when we were getting those weird dictionaries back because our limits were used up?  We won't hold you accountable for handling this inside your function, although it could make some things easier for your own testing.
       
Good luck!

In [36]:
def friend_of_friends_fs(
    names,
    keys = [], 
    to_df = False,
    full_search=True,
    auth=auth
    ):
    
    if len(names) != 2:
        print("You can only enter two names as a list.")
    
    # Checks on screen_name input
    if names[0][0] == '@':
        names[0] = names[0][1:]    

    if names[1][0] == '@':
        names[1] = names[1][1:]       

        
    # Get overall list for first name
    # Go and get results, adding users to an overall list
    friends_list_0 = []
    cursor = -1
    while cursor != 0:
        find_friends_0_url = (
                        'https://api.twitter.com/1.1/friends/list.json'
                         + "?screen_name=" + names[0]
                         + "&cursor=" + str(cursor)
                         + "&count=200" 
                        )
        friends_0_output = requests.get(find_friends_0_url, auth=auth)
        if friends_0_output.status_code == 429:
            print("Reached max API requests.\n"
                  + "Setting cursor to 0 to exit loop.\n"
                  + "friends_0s found so far will be output."
                 )
            cursor = 0
        ##############
        # Modification for function 5
        elif full_search == False:
            cursor = 0 # If have full search turned off set cursor to 0 to exit loop after 1 pass
        ###############
        else:
            friends_0_output_dict = friends_0_output.json()
            cursor = friends_0_output_dict['next_cursor']
            [friends_list_0.append(user) for user in friends_0_output_dict['users']]

            
    # Get overall list for second name
    friends_list_1 = []
    cursor = -1
    while cursor != 0:
        find_friends_1_url = (
                        'https://api.twitter.com/1.1/friends/list.json'
                         + "?screen_name=" + names[1]
                         + "&cursor=" + str(cursor)
                         + "&count=200" 
                        )
        friends_1_output = requests.get(find_friends_1_url, auth=auth)
        if friends_1_output.status_code == 429:
            print("Reached max API requests.\n"
                  + "Setting cursor to 0 to exit loop.\n"
                  + "friends_1s found so far will be output."
                 )
            cursor = 0
        ##############
        # Modification for function 5
        elif full_search == False:
            cursor = 0 # If have full search turned off set cursor to 0 to exit loop after 1 pass
        ###############
        else:
            friends_1_output_dict = friends_1_output.json()
            cursor = friends_1_output_dict['next_cursor']
            [friends_list_1.append(user) for user in friends_1_output_dict['users']]
    
    # Get list of ids for both sets of friends
    name_0_friends_ids = [i['id'] for i in friends_list_0]
    name_1_friends_ids = [i['id'] for i in friends_list_1]

    # Find common ids
    common_friends_list = []
    if len(name_0_friends_ids) < len(name_1_friends_ids):
        common_friend_ids = [i for i in name_0_friends_ids if i in name_1_friends_ids]
        common_friends_list = [i for i in friends_list_0 if i['id'] in common_friend_ids]
        
    else:
        common_friend_ids = [i for i in name_1_friends_ids if i in name_0_friends_ids]
        common_friends_list = [i for i in friends_list_1 if i['id'] in common_friend_ids]
        
    # Limit the friends_list_0 to the specific keys required
    common_friends_list_keys = []
    
    if not keys:
        common_friends_list_keys = common_friends_list
    else:
        for user in common_friends_list:
            temp_dict = {}
            for k in keys:
                temp_dict.update({k:user[k]})
            common_friends_list_keys.append(temp_dict)    
    
    # Put results into a pandas dataframe
    
    
    # Allow for pandas data frame
    if to_df == False:
        return_object = common_friends_list_keys
    elif to_df == True:
        # Loop through each potential column, and add to dictionary
        prep_df_dict = {}
        
        # Check if keys have been designated, if not add them back
        if not keys:
            keys_found = []
            for u in common_friends_list_keys:
                for k in u.keys():
                    keys_found.append(k)
            keys = list(set(keys_found)) # dedupe the keys
            keys.sort()
            
        # Set up the columns
        for k in keys:
            temp_list = [i.get(k,'No value returned.') for i in common_friends_list_keys]
            prep_df_dict.update({k:temp_list})
        
        #Add data to the data frame
        user_df = pd.DataFrame(data=prep_df_dict)
        
        return_object = user_df
    else:
        return_object = 'to_df argument only accpets True or False'
                
    return return_object

In [37]:
test_a = friend_of_friends_fs(['ezraklein', 'tylercowen'], to_df = False,full_search=False)

Reached max API requests.
Setting cursor to 0 to exit loop.
friends_0s found so far will be output.
Reached max API requests.
Setting cursor to 0 to exit loop.
friends_1s found so far will be output.


In [39]:
test_a

[]